In [8]:
#将上述方法提取出来的关键词作为候选关键词，然后考虑每个候选关键词的前后词，根据词性确定是否可以组成候选关键词短语，统计每个簇中
#候选关键词短语的次数，取出现次数最大的候选关键词短语作为关键词短语
#!/usr/bin/python
#-*- encoding:utf-8 -*-
import os
import re
import numpy as np
from numpy import *
import jieba
jieba.load_userdict('/Users/lyj/Desktop/提案程序/sogou.txt')
jieba.load_userdict('/Users/lyj/Desktop/提案程序/标题补充词典.txt')
import jieba.posseg as pseg


#读取stop停用词
def readstopwords(stopwords_path):
    stop_single_words=[]  
    with open(stopwords_path,'r+',encoding='utf-8') as lines:
        for line in lines:  
            content=line.strip()  
            stop_single_words.append(content) 
    return stop_single_words

#分词
def split_words(filePath,stop_single_words):
    f = open(filePath,'r+',encoding='utf-8')
    file_list = f.read()
    f.close()

    lis_before=[] #保存分词结果
    seg_list = pseg.cut(file_list)
    for seg,flag in seg_list: 
        seg = ''.join(seg.split())
        lis_before.append((seg,flag)) 
    
    file_list_after=re.sub('[A-Za-z0-9,\d+(\.\d+)?\%]','',file_list) #去除字母,数字,百分数
    lis_after=[]
    seg_list_after = pseg.cut(file_list_after)
    for seg2,flag2 in seg_list_after: 
        seg2 = ''.join(seg2.split())
        if seg2 not in stop_single_words and len(seg2)!=1:
            lis_after.append((seg2,flag2)) 
    
    return lis_before,lis_after

#将聚类结果分词后，按字典保存
def get_cluster_doc(savepath,stop_single_words):
    cluster_words_dic={}
    cluster_words_dic_after={}
    for i in os.listdir(savepath):
        if i!='.DS_Store':
            words_list=[]
            words_list_after=[]
            path_file = os.path.join(savepath,i)  #取文件路径
            for f in os.listdir(path_file): 
                if f!='.DS_Store':
                    path_file2 =os.path.join(path_file,f)
                    lis_before,lis_after=split_words(path_file2,stop_single_words)
                    words_list.extend(lis_before)
                    words_list_after.extend(lis_after)
            cluster_words_dic[i]=words_list
            cluster_words_dic_after[i]=words_list_after
    return cluster_words_dic,cluster_words_dic_after

#将候选关键词按字典保存
def get_key_words(key_words_savepath):
    key_words_dic={}
    for i in os.listdir(key_words_savepath):
        if i!='.DS_Store':
            f = open(key_words_savepath+"/"+str(i),"r+",encoding="utf-8")
            key_words=f.read().strip().split()
            f.close()
            key_words_dic[i]=key_words
    
    return key_words_dic

# def get_key_phrase(cluster_words_dic,key_words_dic):
#     cluster_word_index_dic={}
#     for key in key_words_dic.keys():
#         cluster_list=[]
#         for key_word in key_words_dic[key]:
#             word_index=[]
#             word_index_dic={}
#             for item in enumerate(cluster_words_dic[key]):
#                 if item[1][0]==key_word:
#                     word_index.append(item[0])
#             word_index_dic[key_word]=word_index
#             cluster_list.append(word_index_dic)
#         cluster_word_index_dic[key]=cluster_list
#     return cluster_word_index_dic

#根据候选关键词的前后词的词性确定是否可以组成关键词短语
def get_key_phrase(cluster_words_dic_after,key_words_dic):
    
    part_speech=['n','nr','ns','nt','nz','vn','an','f']
    key_phrase_dic_dic={}
    key_phrase_list_dic={}
    for key in key_words_dic.keys():
        key_phrase_dic={}
        key_phrase_list=[]
        for key_word in key_words_dic[key]:
            for item in enumerate(cluster_words_dic_after[key]): #enumerate用于查看指定值在列表中的所有位置
                if item[1][0]==key_word:
                    if len(item[1][0])>3:
                        key_phrase_list.append(item[1][0])
                        if item[1][0] not in key_phrase_dic.keys():
                            key_phrase_dic[item[1][0]]=1
                        else:
                            key_phrase_dic[item[1][0]]+=1
                    else:
                        #列表中第一个词，没有前面的词
                        if item[0]==0:
                            if cluster_words_dic_after[key][item[0]+1][1] in part_speech: 
                                key_phrase=item[1][0]+cluster_words_dic_after[key][item[0]+1][0]
                                key_phrase_list.append(key_phrase)
                                if key_phrase not in key_phrase_dic.keys():
                                    key_phrase_dic[key_phrase]=1
                                else:
                                    key_phrase_dic[key_phrase]+=1
                            
                        #列表中最后一个词，没有后面的词
                        elif item[0]==len(cluster_words_dic_after[key])-1:
                            if cluster_words_dic_after[key][item[0]-1][1] in part_speech:
                                key_phrase=cluster_words_dic_after[key][item[0]-1][0]+item[1][0]
                                key_phrase_list.append(key_phrase)
                                if key_phrase not in key_phrase_dic.keys():
                                    key_phrase_dic[key_phrase]=1
                                else:
                                    key_phrase_dic[key_phrase]+=1
                            
                        #列表中中间位置的词
                        else:
                            if cluster_words_dic_after[key][item[0]-1][1] in part_speech and cluster_words_dic_after[key][item[0]+1][1] not in part_speech:
                                key_phrase=cluster_words_dic_after[key][item[0]-1][0]+item[1][0]
                                key_phrase_list.append(key_phrase)
                                if key_phrase not in key_phrase_dic.keys():
                                    key_phrase_dic[key_phrase]=1
                                else:
                                    key_phrase_dic[key_phrase]+=1
                            if cluster_words_dic_after[key][item[0]+1][1] in part_speech and cluster_words_dic_after[key][item[0]-1][1] not in part_speech:
                                key_phrase=item[1][0]+cluster_words_dic_after[key][item[0]+1][0]
                                key_phrase_list.append(key_phrase)
                                if key_phrase not in key_phrase_dic.keys():
                                    key_phrase_dic[key_phrase]=1
                                else:
                                    key_phrase_dic[key_phrase]+=1
                            if cluster_words_dic_after[key][item[0]+1][1] in part_speech and cluster_words_dic_after[key][item[0]-1][1] in part_speech:
                                key_phrase=cluster_words_dic_after[key][item[0]-1][0]+item[1][0]+cluster_words_dic_after[key][item[0]+1][0]
                                key_phrase_list.append(key_phrase)
                                if key_phrase not in key_phrase_dic.keys():
                                    key_phrase_dic[key_phrase]=1
                                else:
                                    key_phrase_dic[key_phrase]+=1
                            
        key_phrase_dic_dic[key]=key_phrase_dic
        key_phrase_list_dic[key]=key_phrase_list
    return key_phrase_dic_dic,key_phrase_list_dic

def compare(key_phrase_dic_before,key_phrase_list_before,key_phrase_dic_after,key_phrase_list_after):
    dic={}
    for key in key_phrase_list_before.keys():
        Intersection=list(set(key_phrase_list_before[key]).intersection(set(key_phrase_list_after[key])))
        for word in key_phrase_dic_after[key].keys():
            if word not in Intersection:
                key_phrase_dic_after[key][word]=-1
        if key=='1.txt':
            dic[1]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='2.txt':
            dic[2]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='3.txt':
            dic[3]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='4.txt':
            dic[4]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='5.txt':
            dic[5]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='6.txt':
            dic[6]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='7.txt':
            dic[7]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
        if key=='8.txt':
            dic[8]=sorted(key_phrase_dic_after[key].items(),key = lambda x:x[1],reverse = True)[:3]
    return dic

stopwords_path="/Users/lyj/Desktop/提案程序/百度停用词列表（去除英文）+tycb_symbol1.txt"
savepath="/Users/lyj/Desktop/提案程序/word2veccluster" #保存聚类结果
key_words_savepath="/Users/lyj/Desktop/提案程序/候选关键词"

stop_single_words=readstopwords(stopwords_path)
cluster_words_dic,cluster_words_dic_after=get_cluster_doc(savepath,stop_single_words) #将聚类后的文档根据是否去除停用词分别按字典保存
key_words_dic=get_key_words(key_words_savepath) #将候选关键词按字典保存
key_phrase_dic_before,key_phrase_list_before=get_key_phrase(cluster_words_dic,key_words_dic) #得到未去除停用词的候选关键词短语
key_phrase_dic_after,key_phrase_list_after=get_key_phrase(cluster_words_dic_after,key_words_dic) #得到去除停用词的候选关键词短语
dic=compare(key_phrase_dic_before,key_phrase_list_before,key_phrase_dic_after,key_phrase_list_after) #两者比较得到最终的关键词短语
for i in sorted(dic.items(),key = lambda x:x[0]):
    word=[]
    for j in i[1]:
        word.append(j[0])
    print("主题"+str(i[0])+"的关键词:",word)
    

主题1的关键词: ['文化遗产', '文化中心', '传统文化']
主题2的关键词: ['知识产权', '科技创新', '科技创新中心']
主题3的关键词: ['轨道交通', '基础设施', '美丽乡村']
主题4的关键词: ['学前教育', '中小学生', '融合教育']
主题5的关键词: ['共享单车', '老年代步车', '物业管理']
主题6的关键词: ['垃圾分类', '资源回收', '建筑垃圾']
主题7的关键词: ['分级诊疗', '医疗机构', '医疗服务']
主题8的关键词: ['污泥利用', '污水处理', '污泥处理']
